In [1]:
import csv
import json

from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [9]:
#1. 기존 NLC CSV 파일을 JSON Object로 변환이 필요합니다.
#     예제 CSV 파일 구조:
#         Text Text Text, label1, label2
#         Text Text Text, label3
#     예제 JSON 구조:
#     [
#         {
#             "text": "document 1",
#             "labels": ["label1"]
#         },
#         {
#             "text": "document 2",
#             "labels": ["label2", "label3"]
#         }
#     ]
def convert_nlc_csv_to_nlu_json(nlc_training_data_file_name):
    
    nlu_data = []

    with open(nlc_training_data_file_name, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            text = row[0]
            labels = row[1:]
            # Convert the text and label in NLU training data JSON object
            data_dict = {
                'text': text,
                'labels': labels
            }
            nlu_data.append(data_dict)

    return nlu_data

csv_data = r'NLC_training_example_v9.csv'
nlu_json_data = convert_nlc_csv_to_nlu_json(csv_data)

#Debug:
json_formatted_str = json.dumps(nlu_json_data, indent=2, ensure_ascii=False)

print(json_formatted_str)

[
  {
    "text": "﻿사과는 절임 잼 빵 등 다양한 요리의 재료로 쓰이는 과일이다. 대표적인 요리는 애플파이. 말 그대로 사과를 넣은 파이로 그 달달한 맛에 중독되면 못 나온다. 미국에서는 가정에서 만들어 먹는 경우도 꽤 된다.",
    "labels": [
      "사과"
    ]
  },
  {
    "text": "사과는 사과나무의 열매로, 세계적으로 널리 재배되는 열매 가운데 하나이다. 평과, 빈파라고도 한다. 사과열매는 가을에 익는데, 보통 지름이 5~9센티미터이다. 극히 드물지만 15센티에 이르기도 한다. 씨앗에는 미량의 사이안화물이 함유되어 있다. 샐러드, 주스, 파이, 카레 등의 재료로 쓰인다.",
    "labels": [
      "사과"
    ]
  },
  {
    "text": "사과나무의 원산지는 발칸반도로 알려져 있으며 B.C. 20세기 경의 스위스 토굴 주거지에서 탄화된 사과가 발굴된 것으로 보아 서양사과는 4,000년 이상의 재배 역사를 가진 것으로 추정된다. 그리스 시대에는 재배종, 야생종을 구분한 기록이 있고 접목 번식법이 이미 소개 되어 있을 정도로 재배 기술이 진보되었다. 로마시대에는 Malus 또는 Malum이란 명칭으로 재배가 성향하였고 그 후 16-17세기에 걸쳐 유럽각지에 전파되었다. 17세기에는 미국에 전파되었고 20세기에는 칠레 등 남미 각국에 전파되었다.",
    "labels": [
      "사과"
    ]
  },
  {
    "text": "사과는 절임 잼 빵 등 다양한 요리의 재료로 쓰이는 과일이다. 대표적인 요리는 애플파이. 말 그대로 사과를 넣은 파이로 그 달달한 맛에 중독되면 못 나온다. 미국에서는 가정에서 만들어 먹는 경우도 꽤 된다.",
    "labels": [
      "사과"
    ]
  },
  {
    "text": "사과는 사과나무의 열매로, 세계적으로 널리 재배되는 열매 가운데 하나이다. 평과, 빈파라고도 한다. 사과열매는 가을에 익는데

In [10]:
#2. json object nlu_data를 파일로 저장 합니다.
def save_json_obj_to_file(jsonFilePath, json_obj):
    with open(jsonFilePath, 'w', encoding='utf-8') as f:
        json.dump(json_obj, f, indent=4)
        
json_nlu_data_fileName = 'nlu_training_data.json'
save_json_obj_to_file(json_nlu_data_fileName, nlu_json_data)

# #Debug: check saved json data
# with open(json_nlu_data_fileName, 'rb') as file:
#     data = json.load(file)
#     for p in data:
#         print('text: ' + p['text'])
#         for d in range(0, len(p['labels'])):
#             print('labels: ' + p['labels'][d])
#         print('')

In [11]:
#3. IAM API Key, NLU 버전, NLU 서비스 URL(생성된 NLU 서비스 화면에 노출 되는 URL)을 지정합니다.
authenticator = IAMAuthenticator('IAM API 키')

nlu = NaturalLanguageUnderstandingV1(
    version='2021-03-25',
    authenticator=authenticator
)

nlu.set_service_url(
    'https://api.kr-seo....NLU 인스턴스 페이지의 URL...')

In [12]:
#4. 사용중인 서비스 모델 확인 (라이트 플랜은 커스텀 모델 1개만 생성 가능하며, 여러번 테스트 하시는 경우 삭제 후 진행시도 필요합니다.)
response = nlu.list_models().get_result()['models']
print(json.dumps(response, indent=2))

#!!!!!!!!!!!!!!!!주의: 실행시 모델을 삭제 합니다!!!!!!!!!!!!!!!!!

#커스텀 모델 존재시 삭제, 아래 커멘트 되어 있는 블럭에 커맨트 해제후 삭제 실행
if(len(response) > 0):
    response = nlu.delete_model(
        model_id=response[0]['model_id']).get_result()
    print(json.dumps(response, indent=2))

#!!!!!!!!!!!!!!!!주의: 실행시 모델을 삭제 합니다!!!!!!!!!!!!!!!!!

[]


In [13]:
#4. json nlu data 파일을 열어 NLU 학습 SDK 호출
def train_nlu_with_nlc_json_file(json_data_fileName):
    model = None
    with open(json_data_fileName, 'rb') as file:
        model = nlu.create_classifications_model(language='ko', training_data=file, training_data_content_type='application/json', name='myNlcDataModel', model_version='1.0.1').get_result()

        print("Created a NLU Classifications model:")
        print(json.dumps(model, indent=4))
    return model

model = train_nlu_with_nlc_json_file(json_nlu_data_fileName)

Created a NLU Classifications model:
{
    "name": "myNlcDataModel",
    "user_metadata": null,
    "language": "ko",
    "description": null,
    "model_version": "1.0.1",
    "version": "1.0.1",
    "workspace_id": null,
    "version_description": null,
    "status": "starting",
    "notices": [],
    "model_id": "f571d903-bc82-4885-89ce-3eefbf918e62",
    "features": [
        "classifications"
    ],
    "created": "2021-09-06T05:24:14Z",
    "last_trained": "2021-09-06T05:24:14Z",
    "last_deployed": null
}


In [14]:
#5. 학습된 NLU 모델이 배포되어야 테스트가 가능합니다. 배포는 약 10~15분 정도 소요됩니다.
#(배포기준: last_deployed 날짜/시간 값이 null이 아닌 경우)
model_id = model['model_id']
model_to_view = nlu.get_classifications_model(model_id=model_id).get_result()

print("Information about the created NLU Classifications model:")
print(json.dumps(model_to_view, indent=4))

#30초마다 모델 조회 및 last_deployed의 null(None) 확인
import time
while True:
    time.sleep(30)
    model_to_view = nlu.get_classifications_model(model_id=model_id).get_result()
    print("model->last_deployed: " + str(model_to_view['last_deployed']))
    if(model_to_view['last_deployed'] != None):
        break

Information about the created NLU Classifications model:
{
    "name": "myNlcDataModel",
    "user_metadata": null,
    "language": "ko",
    "description": null,
    "model_version": "1.0.1",
    "version": "1.0.1",
    "workspace_id": null,
    "version_description": null,
    "status": "starting",
    "notices": [],
    "model_id": "f571d903-bc82-4885-89ce-3eefbf918e62",
    "features": [
        "classifications"
    ],
    "created": "2021-09-06T05:24:14Z",
    "last_trained": "2021-09-06T05:24:14Z",
    "last_deployed": null
}
model->last_deployed: None
model->last_deployed: None
model->last_deployed: None
model->last_deployed: None
model->last_deployed: None
model->last_deployed: None
model->last_deployed: None
model->last_deployed: None
model->last_deployed: None
model->last_deployed: None
model->last_deployed: None
model->last_deployed: None
model->last_deployed: 2021-09-06T05:30:36Z


In [15]:
#6. 테스트 호출:
from ibm_watson.natural_language_understanding_v1 import Features, ClassificationsOptions

text = "바나나는 주로 식용을 목적으로 재배되어, 간식으로 이용되거나 가공 곡식, 과일 칵테일, 샐러드, 케이크, 파이 등에 재료로 쓰인다."

analysis = nlu.analyze(text=text, features=Features(classifications=ClassificationsOptions(model=model_id))).get_result()

print("Analysis response from trained NLU Classifications model:")
print(json.dumps(analysis, indent=4, ensure_ascii=False))

Analysis response from trained NLU Classifications model:
{
    "usage": {
        "text_units": 1,
        "text_characters": 73,
        "features": 1
    },
    "language": "ko",
    "classifications": [
        {
            "confidence": 0.756056,
            "class_name": "바나나"
        },
        {
            "confidence": 0.202394,
            "class_name": "사과"
        },
        {
            "confidence": 0.036488,
            "class_name": "오렌지"
        }
    ]
}


In [16]:
#7. 사용된 모델 삭제

#(아래 커멘트 되어 있는 블럭에 커맨트 해제후 삭제 실행):
#!!!!!!!!!!!!!!!!주의: 실행시 모델을 삭제 합니다!!!!!!!!!!!!!!!!!
response = nlu.delete_model(model_id=model_id).get_result()
print(json.dumps(response, indent=2))
#!!!!!!!!!!!!!!!!주의: 실행시 모델을 삭제 합니다!!!!!!!!!!!!!!!!!

{
  "deleted": "f571d903-bc82-4885-89ce-3eefbf918e62"
}
